In [1]:
from copy import deepcopy
import heapq

In [2]:
class Game:
    def __init__(self, player_points=50, mana=500, boss_points=58, boss_damage=9, part=1):
        self.player_points = player_points
        self.mana = mana
        self.boss_points = boss_points
        self.boss_damage = boss_damage
        self.armor = 0
        self.shield_counter = 0
        self.poison_counter = 0
        self.recharge_counter = 0
        self.missile_active = False
        self.drain_active = False
        self.turn = 'player'
        self.shield_restart = False
        self.poison_restart = False
        self.recharge_restart = False
        self.part = part

    def state(self):
        return (self.player_points, self.mana, self.boss_points, self.armor, self.shield_counter, 
            self.poison_counter, self.recharge_counter, self.missile_active, self.drain_active, self.turn)

    def activate_missile(self):
        self.missile_active = True
        self.mana -= 53

    def activate_drain(self):
        self.drain_active = True
        self.mana -= 73
    
    def activate_shield(self):
        if self.shield_counter == 0:
            self.shield_wait = True
        else:
            self.shield_wait = False
        if self.shield_counter == 1:
            self.shield_restart = True
        self.shield_counter = 6
        self.mana -= 113

    def activate_poison(self):
        if self.poison_counter == 0:
            self.poison_wait = True
        else:
            self.poison_wait = False
        if self.poison_counter == 1:
            self.poison_restart = True
        self.poison_counter = 6
        self.mana -= 173
    
    def activate_recharge(self):
        if self.recharge_counter == 0:
            self.recharge_wait = True
        else:
            self.recharge_wait = False
        if self.recharge_counter == 1:
            self.recharge_restart = True
        self.recharge_counter = 5
        self.mana -= 229

    def apply_effects(self):
        if self.missile_active:
            self.boss_points -= 4
            self.missile_active = False
        if self.drain_active:
            self.boss_points -= 2
            self.player_points += 2
            self.drain_active = False
        if self.shield_counter > 0:
            if not self.shield_wait:
                self.armor = 7
                if not self.shield_restart:
                    self.shield_counter -= 1
                else:
                    self.shield_restart = False
            else:
                self.shield_wait = False
        else:
            self.armor = 0
        if self.poison_counter > 0:
            if not self.poison_wait:
                self.boss_points -= 3
                if not self.poison_restart:
                    self.poison_counter -= 1
                else:
                    self.poison_restart = False
            else:
                self.poison_wait = False
        if self.recharge_counter > 0:
            if not self.recharge_wait:
                self.mana += 101
                if not self.recharge_restart:
                    self.recharge_counter -= 1
                else:
                    self.recharge_restart = False
            else:
                self.recharge_wait = False

    def go_turn(self):
        res = 0
        if self.part == 2 and self.turn == 'player':
            self.player_points -= 1
        self.apply_effects()
        if self.boss_points <= 0:
            res = 1
        if self.turn == 'boss':
            self.player_points -= max(self.boss_damage - self.armor, 1)
            if self.player_points <= 0 and res == 0:
                res = -1
            self.turn = 'player'
        else:
            self.turn = 'boss'
        return res

In [3]:
def next_possible_states(g):
    states = []
    if g.mana >= 53:
        h = deepcopy(g)
        h.activate_missile()
        states.append((h, 53))
    if g.mana >= 73:
        h = deepcopy(g)
        h.activate_drain()
        states.append((h, 73))
    if g.shield_counter <= 1 and g.mana >= 113:
        h = deepcopy(g)
        h.activate_shield()
        states.append((h, 113))
    if g.poison_counter <= 1 and g.mana >= 173:
        h = deepcopy(g)
        h.activate_poison()
        states.append((h, 173))
    if g.recharge_counter <= 1 and g.mana >= 229:
        h = deepcopy(g)
        h.activate_recharge()
        states.append((h, 229))
    return states
    
def run_game(g):
    q = []
    dist = {g.state(): 0}
    order = 0
    explored = set()
    for (v, cost) in next_possible_states(g):
        dist[v.state()] = cost
        heapq.heappush(q, (cost, order, v))
        order += 1
    while q:
        d, _, u = heapq.heappop(q)
        u_state = u.state()
        outcome = u.go_turn()
        if outcome == 1:
            return d
        outcome = u.go_turn()
        if outcome == 1:
            return d
        dist[u.state()] = d
        explored.add(u.state())
        if outcome == 0:
            for (v, cost) in next_possible_states(u):
                if not v.state() in explored:
                    alt = d + cost
                    if v.state() in dist:
                        if alt < dist[v.state()]:
                            dist[v.state()] = alt
                    else:
                        dist[v.state()] = alt
                    heapq.heappush(q, (dist[v.state()], order, v))
                    order += 1
                    explored.add(v.state())
    return 'Not found'


In [4]:
g1 = Game()
part1 = run_game(g1)
g2 = Game(part=2)
part2 = run_game(g2)
print(f'Part 1: {part1}')
print(f'Part 2: {part2}')

Part 1: 1269
Part 2: 1309
